# ANALISE DA FREQUENCIA DOS ADJETIVOS

A proposta aqui é representar o texto em função dos adjetivos encontrados. Ao invés de um bag-of-words ser um bag-of-adjectives. Primeiro testar com todos os adjetivos sem qualquer restrição. Após isso podemos pegar somente os adjetivos mais frequentes, adjetivos exclusivos de uma classe e assim em diante.
Portanto para cada análise, os seguintes passos serão executados;
<ol>
    <li>Normalização do texto: remoção dos acentos, cedilhas, passar letras maiúsculas para minúsculas</li>
    <li>Vetorização do texto</li>
    <li>Remoção da média e setar desvio padrão unitário</li>
    <li>Analise da correlação geral do conjunto de dados</li>
    <li>Analise da correlação dentro de cada classe</li>
    <li>Dividir o conjunto em treino e teste</li>
    <li>Teste dos classificadores: MultinomialNB, SVM e KMeans</li>
</ol>

## Importação das bibliotecas necessárias

In [1]:
import nltk
import pymysql
import numpy as np
import matplotlib.pyplot as plt

from pickle import load
from nltk import FreqDist
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from database_utils import DatabaseConnector, build_dataframe, normalize_text


## Definição de funções úteis

In [2]:
def get_data_from_db(sentiment = None):
    db_connector = DatabaseConnector('localhost', 'root', '12345', 'CORPUS_VIES')
    if sentiment != None:
        retrieved_data = build_dataframe(db_connector.getDataBySentiment(sentiment))
    else:
        retrieved_data = build_dataframe(db_connector.getDataTextAndLabel())

    return retrieved_data

def load_tagger():
    f = open('bigram_tagger.pkl', 'rb')
    tagger = load(f)
    f.close()
    return tagger

def get_all_adjectives():
    corpus = get_data_from_db()
    corpus = corpus['texts'].tolist()
    tagger = load_tagger()
        
    adjectives = set()
    for text in corpus:
        adjectives_found = set([word for (word, tag) in tagger.tag(text.split()) if tag[:3] == 'ADJ'])
        adjectives = adjectives.union(adjectives_found)
        
    print("Numero de adjetivos encontrados: ", len(adjectives))
    return adjectives

def get_adjective_by_sentiment(sentiment):
    lista = get_data_from_db(sentiment= sentiment)

    result_list = []
    for sentence in lista:
        result = tagger.tag(sentence[0].split())
        result_list += result

    fd = FreqDist([word for (word,tag) in result_list if tag[:3] == 'ADJ'])
    adj_set = set(fd.keys())
    print(len(adj_set), ' Adjectives encountered\n')
    
    return adj_set

def adjective_vectorizer(sentence, base_adjectives):
#     normalized_sentence  = normalize_text(sentence)
    normalized_sentence  = sentence
    v = {adjective: normalized_sentence.split().count(adjective) for adjective in base_adjectives}
    return v

def get_correlation_matrix(matrix_data):
    import numpy as np
    import matplotlib.pyplot as plt

    print("Tamanho do corpus: ", matrix_data.shape[0])
    print("Dimensionalidade: ", matrix_data.shape[1])
    
    corr_matrix = np.corrcoef(bag_of_words)
    plt.figure()
    plt.imshow(np.multiply(corr_matrix, 255), cmap='Greys_r')
    plt.show()
    
    return corr_matrix



In [6]:
adj_list = get_all_adjectives()
s = 'processamento de sinais é difícil'
adjective_vectorizer(s, adj_list)

tagger = load_tagger()
print(tagger.tag(["difícil"]))
print(tagger.tag(["dificil"]))

SELECT PARAGRAPH, POLARITY FROM PARAGRAPHS WHERE POLARITY IS NOT NULL AND trim(POLARITY) <> ""
1042  Paragraphs encountered
Numero de adjetivos encontrados:  473
[('difícil', 'ADJ')]
[('dificil', 'NN')]


In [4]:
 import unicodedata
s='é muito Difícil caça'
s = s.lower()
s = ''.join((c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'))
s

'e muito dificil caca'